# Integrative multiomic pipeline

In [1]:
suppressMessages(library(tidyverse))
suppressMessages(library(data.table))
suppressMessages(library(tidymodels))
suppressMessages(library(pROC))
suppressMessages(library(future.apply))

In [2]:
plan(multicore, workers=10)
options(future.globals.maxSize= 30000*1024^2)
options(future.rng.onMisuse = "ignore")

## Pipelie parameters

In [3]:
# fraction of samples used for model training
TRAIN_SPLIT_PROP <- 0.7

# no. of iterations of train-test splits for estimating mean AUROC
N_ITERATIONS <- 200

## Import data

In [4]:
# dataframe with column default (outcome) and one column per SNP
snp_matrix <- readRDS("../../preprocessing/pipeline_input/outputs/snp_matrix.rds")
head(snp_matrix)

,default,rs28619217,200610-10,rs367572771,rs144402189,rs375896687,200610-107,200610-108,200610-109,rs199838004,⋯,rs9999890,rs999993,rs9999931,rs999994,rs9999944,rs999995,rs9999953,rs9999955,rs9999966,rs9999992
,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
9323111011_R01C01.Theta,Yes,0.05048173,0.03499985,0.8267820,0.07534563,0.02177286,0.02820558,0.04560787,0.01812311,0.03443542,⋯,0.03955950,0.02094064,0.03444201,0.9566568,0.9893615,0.05627595,0.9744269,0.9546334,0.9496952,0.9727035
9323111011_R02C01.Theta,Yes,0.04993737,0.03245308,0.8402165,0.05894551,0.01899610,0.02572266,0.03968269,0.01453403,0.03323122,⋯,0.04284558,0.56658630,0.64182480,0.9567050,0.9963050,0.98109540,0.9907560,0.5683809,0.9541447,0.5788528
9323111011_R03C01.Theta,Yes,0.05869108,0.03736536,0.8192294,0.05012877,0.02201855,0.02720243,0.04521938,0.01640155,0.03577631,⋯,0.05252435,0.02227613,0.95534770,0.9556491,0.9896312,0.04707525,0.9817939,0.9627419,0.9641120,0.9697003
9323111011_R04C01.Theta,Yes,0.06344332,0.03846529,0.8327740,0.04648601,0.02907989,0.02472522,0.04957985,0.02264668,0.04223788,⋯,0.05119100,0.02344536,0.96338160,0.9538175,0.9944138,0.49768020,0.9743326,0.5918900,0.4754377,0.9762244
9323111011_R05C01.Theta,Yes,0.05447656,0.03445135,0.8324282,0.06610103,0.01693385,0.03079994,0.83544770,0.01500036,0.03983220,⋯,0.02498142,0.02266808,0.59204600,0.9606557,0.5812460,0.46217830,0.9791297,0.5576898,0.9642391,0.5552713
9323111011_R06C01.Theta,Yes,0.05176229,0.03717544,0.8438321,0.06748527,0.02644398,0.02868403,0.04987841,0.01938164,0.04129269,⋯,0.06561337,0.01662327,0.97650020,0.9538698,0.9888092,0.05456477,0.9762215,0.5760989,0.9659798,0.9717936


In [5]:
# dataframe with columns SNP, GWAS P-value, and 10 annotation columns
snp_metadata <- as.data.frame(fread("../../preprocessing/pipeline_input/outputs/snp_metadata.txt"))
head(snp_metadata)

,SNP,P,H3K27ac,H3K27me3,H3K4me1,H3K4me2,H3K4me3,H3K36me3,non_neuroectoderm,epidermal,placode,neural_crest
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,rs4475691,0.84230,0,0,119.6667,0,0,0.00000,0,69.56308,0,0
2,rs6660139,0.59800,0,0,0.0000,0,0,0.00000,0,69.56308,0,0
3,rs3128108,0.37530,0,0,0.0000,0,0,0.00000,0,69.56308,0,0
4,rs3128117,0.07298,0,0,0.0000,0,0,0.00000,0,69.56308,0,0
5,rs115413462,0.89460,0,0,0.0000,0,0,68.22222,0,69.56308,0,0
6,rs6671356,0.64750,0,0,0.0000,0,0,0.00000,0,69.56308,0,0


## Core Functions

In [6]:
# scale a numeric to the range of 100
# with a pseudocount of +1 to avoid zero values
scale_with_pseudocount <- function(x) {
    ((x - min(x)) / (max(x) - min(x))) * 100 + 1
}

In [7]:
select_top_snps <- function(snp_metadata, snp_matrix) {

    scaled_metadata <- snp_metadata

    # scale the gwas p-values
    scaled_metadata$P <- scale_with_pseudocount(-log10(scaled_metadata$P))

    has_annotations <- ncol(scaled_metadata) > 2

    if (has_annotations) {
        # scale each annotation column 
        # then combine all scores by product        
        for (i in 3:ncol(scaled_metadata)) {
            scaled_metadata[, i] <- scale_with_pseudocount(scaled_metadata[, i])
        }
        scaled_metadata$Score <- apply(scaled_metadata[, 2:ncol(scaled_metadata)], 1, prod)
    } else {
        # if no additional annotation, use scaled p-value as the score directly
        scaled_metadata$Score <- scaled_metadata$P
    }

    # select top n SNPs to maintain a 1:5 feature-to-sample ratio
    # reducing the risk of overfitting
    snp_scores          <- scaled_metadata[, c("SNP", "Score")]
    snp_scores_filtered <- snp_scores[snp_scores$SNP %chin% colnames(snp_matrix[, -1]), ]

    top_n    <- nrow(snp_matrix) %/% 5
    top_snps <- snp_scores_filtered |>
        arrange(desc(Score)) |>
        head(top_n) |>
        pull(SNP)

    return(top_snps)
}

In [8]:
# fits a logistic regression model over 200 iterations of 7:3 train-test splits 
# returns the mean AUROC across all iterations.

compute_mean_auc <- function(model_input) {

    auc_values <- lapply(1:N_ITERATIONS, function(i) {

        if (i %% 100 == 0) message("  Iteration ", i, " / ", N_ITERATIONS)

        # 7:3 train-test split
        split      <- initial_split(model_input, prop = TRAIN_SPLIT_PROP, strata = default)
        train_data <- training(split)
        test_data  <- testing(split)

        # Fit logistic regression model
        lr_fit <- logistic_reg(mixture = 0, penalty = 0) |>
            set_engine("glmnet") |>
            set_mode("classification") |>
            fit(default ~ ., data = train_data)

        # Predict probabilities on test set
        predictions <- predict(lr_fit, new_data = test_data, type = "prob") |>
            as.data.frame() |>
            mutate(default = test_data$default) |>
            select(-.pred_No)

        # Compute AUC for this iteration
        auc(roc(
            response  = predictions$default,
            predictor = predictions$.pred_Yes,
            direction = "<",
            levels    = c("No", "Yes")
        ))
    })

    return(mean(as.numeric(auc_values)))
}

In [9]:
# Run the full SNP selection -> Model training -> mean AUROC calculation pipeline

run_pipeline <- function(snp_metadata, snp_matrix) {
    top_snps    <- select_top_snps(snp_metadata, snp_matrix)
    model_input <- snp_matrix[, c("default", top_snps)]
    compute_mean_auc(model_input)
}

In [10]:
# evaluate all combinations of annotation layers
# by generating every possible subset of the annotation columns in snp_metadata (columns 3 onward) 
# then runs the full pipeline (select_top_snps -> compute_mean_auc) for each subset

run_all_combinations <- function(snp_metadata, snp_matrix) {

    annotation_layers <- colnames(snp_metadata)[3:ncol(snp_metadata)]
    annotation_combos <- unlist(
        lapply(seq_along(annotation_layers), combn, x = annotation_layers, simplify = FALSE),
        recursive = FALSE
    )

    # also add the p-value-only baseline as a combo with no annotation columns
    all_combinations <- c(list(character(0)), annotation_combos)

    message("Running ", length(all_combinations), " combinations (including p-value-only baseline) in parallel...")

    results <- future_lapply(all_combinations, function(combo) {
        combo_cols <- unlist(combo)
        auc_value  <- run_pipeline(
            snp_metadata[, c("SNP", "P", combo_cols)],
            snp_matrix
        )
        label <- if (length(combo_cols) == 0) "GWAS" else paste0("GWAS&", paste0(combo_cols, collapse = "&"))
        c(label, auc_value)
    })

    output             <- as.data.frame(do.call(rbind, results))
    colnames(output)   <- c("Combination", "AUC")
    output$Combination <- unlist(output$Combination)
    output$AUC         <- as.numeric(unlist(output$AUC))

    return(output)
}

## Run pipeline

In [11]:
auroc_all_comb <- run_all_combinations(snp_metadata, snp_matrix)

Running 1024 combinations (including p-value-only baseline) in parallel...

  Iteration 100 / 200

  Iteration 200 / 200

  Iteration 100 / 200

  Iteration 200 / 200

  Iteration 100 / 200

  Iteration 200 / 200

  Iteration 100 / 200

  Iteration 200 / 200

  Iteration 100 / 200

  Iteration 200 / 200

  Iteration 100 / 200

  Iteration 200 / 200

  Iteration 100 / 200

  Iteration 200 / 200

  Iteration 100 / 200

  Iteration 200 / 200

  Iteration 100 / 200

  Iteration 200 / 200

  Iteration 100 / 200

  Iteration 200 / 200

  Iteration 100 / 200

  Iteration 200 / 200

  Iteration 100 / 200

  Iteration 200 / 200

  Iteration 100 / 200

  Iteration 200 / 200

  Iteration 100 / 200

  Iteration 200 / 200

  Iteration 100 / 200

  Iteration 200 / 200

  Iteration 100 / 200

  Iteration 200 / 200

  Iteration 100 / 200

  Iteration 200 / 200

  Iteration 100 / 200

  Iteration 200 / 200

  Iteration 100 / 200

  Iteration 200 / 200

  Iteration 100 / 200

  Iteration 200 / 200

  It

In [12]:
write.table(arrange(auroc_all_comb, desc(AUC)), file = "outputs/auroc_all_combinations.txt", 
            sep = "\t", col.names = T, row.names = F, quote = F)